# BLP Homework Simulation

This notebook generates fake data and solves for equilibrium prices as in the homework assignment.

In [4]:
import numpy as np, pandas as pd
from scipy.optimize import root
from scipy.linalg import cholesky

# Reproducibility
np.random.seed(1995)

# Parameters
T = 600   # markets (demo; adjust to 600 for full run)
J = 4    # products per market
R = 1000  # consumer draws

beta1 = 1.0
alpha = -2.0
gamma0 = 0.5
gamma1 = 0.25

# Draw x, w
x = np.abs(np.random.randn(T, J))
w = np.abs(np.random.randn(T, J))

# Draw (xi, omega)
Sigma = np.array([[1.0, 0.25],[0.25, 1.0]])
A = cholesky(Sigma, lower=True)
z = np.random.randn(T*J, 2)
draws = z.dot(A.T)
xi = draws[:,0].reshape(T, J)
omega = draws[:,1].reshape(T, J)

# Random coefficients
beta2_draws = np.random.randn(R) + 4.0
beta3_draws = np.random.randn(R) + 4.0
sat_indicator = np.array([1,1,0,0])
wired_indicator = np.array([0,0,1,1])

def market_shares_and_derivatives(t, p):
    # Step 1: Compute choice probabilities conditional on random coefficients
    bx = beta1 * x[t].reshape(1,J)
    b2 = beta2_draws.reshape(R,1) * sat_indicator.reshape(1,J)
    b3 = beta3_draws.reshape(R,1) * wired_indicator.reshape(1,J)
    ap = alpha * p.reshape(1,J)
    xi_mat = xi[t].reshape(1,J)
    V = bx + b2 + b3 + ap + xi_mat
    expV = np.exp(V)
    denom = 1.0 + expV.sum(axis=1, keepdims=True)
    P = expV / denom  # P[r,j] = s_jt conditional on draw r
    
    # Step 2: s_jt = (1/R) sum_r s_jt|r
    s = P.mean(axis=0)
    
    # Step 3: Approximate ∂s_jt/∂p_kt using derivative of integrand
    # ∂s_jt|r / ∂p_kt = α * s_jt|r * (δ_{jk} - s_kt|r)
    # So ∂s_jt/∂p_kt ≈ α * (1/R) sum_r s_jt|r * (δ_{jk} - s_kt|r)
    # = α/R * [sum_r s_jt|r δ_{jk} - sum_r s_jt|r s_kt|r]
    # = α/R * [s_jt * R - (P[:,j] * P[:,k]).sum()]
    # Since sum_r s_jt|r = R s_jt, and (P.T @ P)[j,k] = sum_r P[r,j] P[r,k]
    sum_diag = np.sum(P, axis=0)  # R * s
    sum_outer = P.T.dot(P)  # sum_r P[r,j] P[r,k]
    deriv = alpha * (np.diag(sum_diag) - sum_outer) / R
    return s, deriv, P

lnmc = gamma0 + w*gamma1 + (omega/8.0)
mc = np.exp(lnmc)

prices = np.zeros((T,J))
shares = np.zeros((T,J))
converged_count = 0

for t in range(T):
    p0 = mc[t] 
    def rootfun(p):
        # FOC: p_j - mc_j = - (∂s_j/∂p_j)^{-1} s_j
        # So p - mc + (∂s/∂p)^{-1} s = 0
        s,D, _ = market_shares_and_derivatives(t,p)
        try:
            invD = np.linalg.inv(D)
        except np.linalg.LinAlgError:
            invD = np.linalg.inv(D+1e-8*np.eye(J))
        return p - mc[t] + invD.dot(s)
    sol = root(rootfun, p0, method='hybr', tol=1e-8, options={'maxfev':1000})
    if sol.success:
        converged_count += 1
    else:
        print(f"Market {t}: {sol.message}")
    prices[t] = sol.x
    shares[t], _, _ = market_shares_and_derivatives(t, sol.x)

print(f"Converged markets: {converged_count}/{T}")
print("Avg prices:", prices.mean(axis=0))
print("Avg shares:", shares.mean(axis=0))

# Now implement Morrow-Skerlos algorithm as per PyBLP documentation
def solve_prices_morrow_skerlos(t, mc_market, max_iter=100, tol=1e-8):
    p = mc_market.copy()  # start from mc
    for it in range(max_iter):
        s, D, P = market_shares_and_derivatives(t, p)
        # Compute Lambda and Gamma
        # Lambda_jj ≈ α * s_j
        Lambda = np.diag(alpha * s)
        # Gamma_jk ≈ α * (P.T @ P)_jk / R
        # But we have sum_outer = P.T @ P, so Gamma = alpha * sum_outer / R
        sum_outer = (P.T @ P)  # from the function, but wait, in the function we have sum_outer = P.T.dot(P), and deriv = alpha * (diag(sum_diag) - sum_outer) / R
        # sum_diag = sum P axis 0 = R * s
        # sum_outer = P.T @ P
        # So Gamma_jk ≈ α * (P.T @ P)_jk / R = α * sum_outer_jk / R
        Gamma = alpha * sum_outer / R
        # zeta = Lambda^{-1} @ (Gamma @ (p - mc) + s)
        diff = p - mc_market
        zeta = np.linalg.inv(Lambda) @ (Gamma @ diff - s)
        p_new = mc_market + zeta
        p_new = np.maximum(p_new, 0.01)  # prevent negative
        if np.max(np.abs(p_new - p)) < tol:
            return p_new, it + 1
        p = p_new
    return p, max_iter

prices_ms = np.zeros((T, J))
iterations_ms = np.zeros(T, dtype=int)
converged_ms = 0

for t in range(T):
    p_ms, iters = solve_prices_morrow_skerlos(t, mc[t])
    prices_ms[t] = p_ms
    iterations_ms[t] = iters
    if iters < 100:  # converged within max_iter
        converged_ms += 1
    else:
        print(f"Market {t}: MS did not converge within {iters} iterations")

shares_ms = np.zeros((T,J))
for t in range(T):
    shares_ms[t], _, _ = market_shares_and_derivatives(t, prices_ms[t])

print(f"Morrow-Skerlos converged markets: {converged_ms}/{T}")
print("Avg iterations for MS:", iterations_ms.mean())
print("Avg prices MS:", prices_ms.mean(axis=0))
print("Avg shares MS:", shares_ms.mean(axis=0))

# Compare prices
price_diff = np.abs(prices - prices_ms)
max_diff = np.max(price_diff)
mean_diff = np.mean(price_diff)
print(f"Max price difference: {max_diff}")
print(f"Mean price difference: {mean_diff}")

if max_diff > 1e-6:
    print("Prices differ significantly between methods.")
else:
    print("Prices are essentially the same.")

# Save dataset
rows = []
for t in range(T):
    for j in range(J):
        rows.append({
            'market': t,
            'product': j+1,
            'x': x[t,j],
            'w': w[t,j],
            'xi': xi[t,j],
            'omega': omega[t,j],
            'mc': mc[t,j],
            'price': prices[t,j],
            'share': shares[t,j],
            'satellite': int(sat_indicator[j]),
            'wired': int(wired_indicator[j])
        })
df = pd.DataFrame(rows)
df.to_csv('blp_fake_dataset.csv', index=False)
print("Dataset saved to blp_fake_dataset.csv")
df.head()


Converged markets: 600/600
Avg prices: [3.34161286 3.28928785 3.32216566 3.32642634]
Avg shares: [0.1325922  0.13937477 0.14087173 0.13062252]
Morrow-Skerlos converged markets: 600/600
Avg iterations for MS: 12.756666666666666
Avg prices MS: [3.34161286 3.28928785 3.32216566 3.32642633]
Avg shares MS: [0.1325922  0.13937477 0.14087173 0.13062252]
Max price difference: 2.514073838000286e-09
Mean price difference: 4.580853254128788e-10
Prices are essentially the same.
Dataset saved to blp_fake_dataset.csv


,market,product,x,w,xi,omega,mc,price,share,satellite,wired
0,0,1,1.240633,0.919340,-0.629390,0.676181,2.257725,3.470859,0.051819,1,0
1,0,2,1.470579,2.068400,1.005377,-1.620066,2.258254,3.471388,0.334093,1,0
2,0,3,2.101191,0.008831,-2.595259,0.954227,1.861693,2.993255,0.054866,0,1
3,0,4,1.464822,2.114361,-0.403500,-1.056475,2.451085,3.582647,0.079957,0,1
4,1,1,0.817922,1.106436,0.846328,0.645967,2.356918,3.589657,0.131969,1,0
